# Titanic data set

Using the Titanic data sets to do predictions about survivability of pasenger based on the columns Pclass, Sex, Age and Fare. 

## Accessing data from file

In [134]:
import pandas as pd

In [135]:
df_titanic = pd.read_csv('csv_files/t9_titanic.csv')
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Getting rid of unwanted columns from Data

In [136]:
df_titanic = df_titanic.drop(['PassengerId','Name','SibSp','Parch',
                                'Ticket','Cabin','Embarked'],
                                 axis='columns')

In [137]:
df_titanic.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


## Creating input and output variables for model

In [138]:
df_titanic_inputs = df_titanic.drop('Survived',axis='columns')
df_titanic_inputs.head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


In [139]:
df_titanic_target = df_titanic['Survived']
df_titanic_target.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

## Using Label Encoding to turn sex into numeric data

In [140]:
from sklearn.preprocessing import LabelEncoder
le_sex = LabelEncoder()
df_titanic_inputs['sex_n'] = le_sex.fit_transform(df_titanic_inputs['Sex'])

In [141]:
df_titanic_inputs.head()

,Pclass,Sex,Age,Fare,sex_n
0,3,male,22.0,7.2500,1
1,1,female,38.0,71.2833,0
2,3,female,26.0,7.9250,0
3,1,female,35.0,53.1000,0
4,3,male,35.0,8.0500,1


In [142]:
df_titanic_inputs_n = df_titanic_inputs.drop('Sex',axis='columns')

In [143]:
df_titanic_inputs_n.head()

,Pclass,Age,Fare,sex_n
0,3,22.0,7.2500,1
1,1,38.0,71.2833,0
2,3,26.0,7.9250,0
3,1,35.0,53.1000,0
4,3,35.0,8.0500,1


## Getting rid of empty cells in the data

Some data missing from the 'Age' column. Will replace the empty cells with average age of all passengers.

In [144]:
import math

In [145]:
mean_age = math.floor(df_titanic_inputs['Age'].mean())
mean_age

29

In [146]:
df_titanic_inputs_n.Age = df_titanic_inputs.Age.fillna(mean_age)

In [147]:
df_titanic_inputs_n.head(10)

,Pclass,Age,Fare,sex_n
0,3,22.0,7.2500,1
1,1,38.0,71.2833,0
2,3,26.0,7.9250,0
3,1,35.0,53.1000,0
4,3,35.0,8.0500,1
5,3,29.0,8.4583,1
6,1,54.0,51.8625,1
7,3,2.0,21.0750,1
8,3,27.0,11.1333,0
9,2,14.0,30.0708,0


## Analysing data to know about the class' fares

In [148]:
df_titanic_1st_class = df_titanic.loc[df_titanic['Pclass'] == 1]
df_titanic_2nd_class = df_titanic.loc[df_titanic['Pclass'] == 2]
df_titanic_3rd_class = df_titanic.loc[df_titanic['Pclass'] == 3]

In [149]:
df_titanic_3rd_class.Survived.mean()

0.24236252545824846

## Separate data into training set and testing set

In [150]:
from sklearn.model_selection import train_test_split

In [151]:
x_train,x_test, y_train,y_test = train_test_split(df_titanic_inputs_n,
                                                  df_titanic_target,
                                                  test_size=0.2, 
                                                  random_state=10)

In [152]:
y_train.head()

57     0
717    1
431    1
633    0
163    0
Name: Survived, dtype: int64

## Creating model using Decision Tree

In [153]:
from sklearn import tree
model_titanic = tree.DecisionTreeClassifier()

In [154]:
model_titanic.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

## Doing Predictions using model that was created

order of columns: 'Pclass' - 'Age' - 'Fare' - 'sex_n'

sex: male=1/ female=0

fares=  1st:84, 2nd:20 , 3rd:13

#### Test Data 1: 1st Class - 55 yrs old - 100usd Fare - Female

In [155]:
model_titanic.predict([[1,55,100,0]])

array([1])

#### Test Data 2: 3rd Class - 21 yrs old - 14 usd Fare - Male

In [160]:
model_titanic.predict([[3,21,14,1]])

array([0])

## Testing Score of Model using test data

In [162]:
model_titanic.score(x_test, y_test)

0.8044692737430168